In [ ]:
import torch
import deepxde as dde
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------------------------------------------------
# 1. パラメータ設定
# ------------------------------------------------------------------------------
LENGTH = 100.0
WIDTH = 10.0
HEIGHT = 10.0
E = 2.1e5
nu = 0.3
mu = E / (2 * (1 + nu))
lmbd = E * nu / ((1 + nu) * (1 - 2 * nu))

LOAD_FORCE = 1000.0
AREA = WIDTH * HEIGHT
TRACTION_X = LOAD_FORCE / AREA  # 10 MPa

# ★ Lossのバランスを取るためのスケーリング係数
# 応力の桁(10^1 MPa) と PDEの桁を合わせるため
SIGMA_SCALE = 10.0 

# ★ 学習回数を増やす
ITERATIONS_ADAM = 15000 
ITERATIONS_LBFGS = 5000

# ------------------------------------------------------------------------------
# 2. 物理方程式の定義
# ------------------------------------------------------------------------------
def compute_stress(x, u):
    du_x = dde.grad.jacobian(u, x, i=0, j=0)
    du_y = dde.grad.jacobian(u, x, i=0, j=1)
    du_z = dde.grad.jacobian(u, x, i=0, j=2)
    dv_x = dde.grad.jacobian(u, x, i=1, j=0)
    dv_y = dde.grad.jacobian(u, x, i=1, j=1)
    dv_z = dde.grad.jacobian(u, x, i=1, j=2)
    dw_x = dde.grad.jacobian(u, x, i=2, j=0)
    dw_y = dde.grad.jacobian(u, x, i=2, j=1)
    dw_z = dde.grad.jacobian(u, x, i=2, j=2)

    E_xx = du_x
    E_yy = dv_y
    E_zz = dw_z
    E_xy = 0.5 * (du_y + dv_x)
    E_yz = 0.5 * (dv_z + dw_y)
    E_zx = 0.5 * (dw_x + du_z)

    trace_E = E_xx + E_yy + E_zz
    s_xx = lmbd * trace_E + 2 * mu * E_xx
    s_yy = lmbd * trace_E + 2 * mu * E_yy
    s_zz = lmbd * trace_E + 2 * mu * E_zz
    s_xy = 2 * mu * E_xy
    s_yz = 2 * mu * E_yz
    s_zx = 2 * mu * E_zx
    
    return s_xx, s_yy, s_zz, s_xy, s_yz, s_zx

def pde(x, u):
    s_xx, s_yy, s_zz, s_xy, s_yz, s_zx = compute_stress(x, u)
    
    s_xx_x = dde.grad.jacobian(s_xx, x, i=0, j=0)
    s_xy_y = dde.grad.jacobian(s_xy, x, i=0, j=1)
    s_zx_z = dde.grad.jacobian(s_zx, x, i=0, j=2)
    
    s_xy_x = dde.grad.jacobian(s_xy, x, i=0, j=0)
    s_yy_y = dde.grad.jacobian(s_yy, x, i=0, j=1)
    s_yz_z = dde.grad.jacobian(s_yz, x, i=0, j=2)
    
    s_zx_x = dde.grad.jacobian(s_zx, x, i=0, j=0)
    s_yz_y = dde.grad.jacobian(s_yz, x, i=0, j=1)
    s_zz_z = dde.grad.jacobian(s_zz, x, i=0, j=2)
    
    momentum_x = s_xx_x + s_xy_y + s_zx_z
    momentum_y = s_xy_x + s_yy_y + s_yz_z
    momentum_z = s_zx_x + s_yz_y + s_zz_z
    
    # 残差をスケーリング
    return [momentum_x / SIGMA_SCALE, momentum_y / SIGMA_SCALE, momentum_z / SIGMA_SCALE]

# ------------------------------------------------------------------------------
# 3. 幾何形状と境界条件 (修正版)
# ------------------------------------------------------------------------------
geom = dde.geometry.Cuboid([0, 0, 0], [LENGTH, WIDTH, HEIGHT])

def boundary_loaded(x, on_boundary):
    return on_boundary and np.isclose(x[0], LENGTH)

# ★【修正】compute_stressには 第1引数の x (Tensor) を渡す
def boundary_loaded_stress_xx(x, y, X):
    s_xx, _, _, _, _, _ = compute_stress(x, y)  # X -> x に変更
    return (s_xx - TRACTION_X) / SIGMA_SCALE

def boundary_loaded_stress_xy(x, y, X):
    _, _, _, s_xy, _, _ = compute_stress(x, y)  # X -> x に変更
    return s_xy / SIGMA_SCALE

def boundary_loaded_stress_xz(x, y, X):
    _, _, _, _, _, s_zx = compute_stress(x, y)  # X -> x に変更
    return s_zx / SIGMA_SCALE

bc_load_xx = dde.OperatorBC(geom, boundary_loaded_stress_xx, boundary_loaded)
bc_load_xy = dde.OperatorBC(geom, boundary_loaded_stress_xy, boundary_loaded)
bc_load_xz = dde.OperatorBC(geom, boundary_loaded_stress_xz, boundary_loaded)

bcs = [bc_load_xx, bc_load_xy, bc_load_xz]
# ------------------------------------------------------------------------------
# 4. モデル構築
# ------------------------------------------------------------------------------
# ★ 改善点1: サンプリング数を増やす（特に境界）
data = dde.data.PDE(
    geom, pde, bcs,
    num_domain=5000,    # 2000 -> 5000
    num_boundary=2000,  # 400 -> 2000 (境界条件を正確に捉えるため)
    num_test=1000
)

# ネットワーク
layer_size = [3] + [64] * 5 + [3] # 深さと幅を少し強化
activation = "tanh"
initializer = "Glorot normal"
net = dde.nn.FNN(layer_size, activation, initializer)

# 入力正規化
net.apply_feature_transform(lambda x: (x - 0) / (LENGTH - 0))

# Hard Constraint + 出力スケーリング
# x=0でu=0を強制する構造
def output_transform(x, y):
    # u ~ 0.005mm 程度なので、yが0.5程度になるよう 0.01 を掛ける
    return (x[:, 0:1] / LENGTH) * y * 0.01

net.apply_output_transform(output_transform)

model = dde.Model(data, net)

# ------------------------------------------------------------------------------
# 5. 学習実行
# ------------------------------------------------------------------------------
# ★ 改善点2: 損失の重み付け (Loss Weights)
# PDE内部残差(3つ)と、境界条件(3つ)のバランスを取ります。
# 境界条件(Neumann)の方を強く強制しないと、変位ゼロ(自明解)に陥りやすいため、
# 境界条件の重みを大きく(x10 ~ x100)します。
loss_weights = [1, 1, 1, 100, 100, 100]

print("--- Adam Training (Phase 1) ---")
# ★ 改善点3: 学習率の減衰 (Learning Rate Decay)
# 1e-3 から初めて、徐々に小さくして精度を高めます
model.compile("adam", lr=1e-3, loss_weights=loss_weights, decay=("inverse time", 2000, 0.9))
losshistory, train_state = model.train(iterations=ITERATIONS_ADAM, display_every=1000)

print("--- L-BFGS Training (Phase 2) ---")
# 仕上げにL-BFGSを使用（重みは維持）
model.compile("L-BFGS", loss_weights=loss_weights)
losshistory, train_state = model.train(display_every=200)

# ------------------------------------------------------------------------------
# 6. 評価
# ------------------------------------------------------------------------------
delta_theory = (LOAD_FORCE * LENGTH) / (AREA * E)

# 予測値
sample_point = np.array([[LENGTH, WIDTH/2, HEIGHT/2]])
u_pred_point = model.predict(sample_point)
u_x_pred = u_pred_point[0, 0]

print("\n================ EVALUATION ================")
print(f"Theoretical Displacement: {delta_theory:.6f} mm")
print(f"Predicted Displacement:   {u_x_pred:.6f} mm")
error_percent = abs(u_x_pred - delta_theory)/delta_theory * 100
print(f"Error: {error_percent:.2f}%")

# 可視化
try:
    x_line = np.linspace(0, LENGTH, 100).reshape(-1, 1)
    y_line = np.full_like(x_line, WIDTH/2)
    z_line = np.full_like(x_line, HEIGHT/2)
    X_line = np.hstack((x_line, y_line, z_line))

    u_line = model.predict(X_line)
    u_x_line = u_line[:, 0]
    u_theory_line = (x_line / LENGTH) * delta_theory

    plt.figure(figsize=(10, 6))
    plt.plot(x_line, u_x_line, 'b-', linewidth=2, label='PINN Prediction')
    plt.plot(x_line, u_theory_line, 'r--', linewidth=2, label='Theoretical Solution')
    plt.title(f'Displacement $u_x$ along Length (Error: {error_percent:.2f}%)')
    plt.xlabel('Position x (mm)')
    plt.ylabel('Displacement u_x (mm)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

except Exception as e:
    print(e)

--- Adam Training (Phase 1) ---
Compiling model...
'compile' took 0.000221 s

Training model...

Step      Train loss                                                      Test loss                                                       Test metric
0         [5.20e-06, 8.09e-06, 9.82e-05, 1.43e+02, 7.74e+00, 3.82e+01]    [5.22e-06, 8.11e-06, 9.84e-05, 1.43e+02, 7.74e+00, 3.82e+01]    []  
